In [52]:
import numpy as np 
import scipy
from scipy.io import arff
import pandas as pd
import tensorflow as tf 
import random 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, cohen_kappa_score, hamming_loss
random.seed = 0

In [127]:
emotions_data_train = scipy.io.arff.loadarff("/home/karim/Documents/research/MLML datasets/emotions/emotions-train.arff")
emotions_data_test = scipy.io.arff.loadarff("/home/karim/Documents/research/MLML datasets/emotions/emotions-test.arff")
features_labels = emotions_data_train[0]
features_labels_df = pd.DataFrame(features_labels)
print("Number of training samples is: {}".format(len(emotions_data[0])))

features_labels_test = emotions_data_test[0]
features_labels_test_df = pd.DataFrame(features_labels_test)
print("Number of test samples is: {}".format(len(features_labels__test_df)))
features = features_labels_df.values[:,:-6]
labels = features_labels_df.values[:,-6:]
test_features = features_labels_test_df.values[:,:-6]
test_labels = features_labels_test_df.values[:,-6:]
LABEL_LIST = list(features_labels_df.columns[-6:])

Number of training samples is: 391
Number of test samples is: 202


In [367]:
input_shape = 72
output_shape = 6
hidden_layer_1_shape = 48
hidden_layer_2_shape = 24

def get_weights(shape):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    #variable_summaries(w)
    return w

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    b = tf.Variable(initial)
    #variable_summaries(b)
    return b

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = get_weights([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

def weighted_loss(y_true, y_pred, positive_weights, negative_weights):
    # clip to prevent NaN's and Inf's
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7, name=None)
    #y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = (-y_true * tf.log(y_pred) * positive_weights) - ((1.0 - y_true) * tf.log(1.0 - y_pred) * negative_weights)
    loss = tf.reduce_mean(loss)
    return loss

In [368]:
# Define a 2 layers network to train 
y = tf.placeholder(tf.float32, [None, output_shape], name="true_labels")
x_input = tf.placeholder(tf.float32, [None,input_shape],name="input_layer")
h1 = tf.nn.relu(full_layer(x_input, hidden_layer_1_shape))
h2 = tf.nn.relu(full_layer(h1, hidden_layer_2_shape))
#h3 = tf.nn.relu(full_layer(h2, hidden_layer_2_shape))
#h4 = tf.nn.relu(full_layer(h3, hidden_layer_2_shape))
logits = full_layer(h2,output_shape)
output = tf.nn.sigmoid(logits)

## Train with original labels [no missing] 

In [369]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
# Learning rate decay
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate=0.1, global_step=global_step, decay_steps=1000,
                                          decay_rate=0.95,staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
correct_prediction = tf.equal(tf.round(output), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [137]:
NUM_EPOCHS = 50000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels,})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), 
                  "Test accuracy: {:.4f}".format(val_accuracies), "Learning rate: {}".format(current_learning_rate))

Epoch #500 Loss: 0.5808 accuracy: 0.7097 Test loss: 0.6398 Test accuracy: 0.6724 Learning rate: 0.10000000149011612
Epoch #1000 Loss: 0.5807 accuracy: 0.7025 Test loss: 0.5929 Test accuracy: 0.6972 Learning rate: 0.0949999988079071
Epoch #1500 Loss: 0.6254 accuracy: 0.6803 Test loss: 0.6060 Test accuracy: 0.6700 Learning rate: 0.0949999988079071
Epoch #2000 Loss: 0.5885 accuracy: 0.7110 Test loss: 0.6079 Test accuracy: 0.6790 Learning rate: 0.09025000035762787
Epoch #2500 Loss: 0.5738 accuracy: 0.7229 Test loss: 0.6027 Test accuracy: 0.6947 Learning rate: 0.09025000035762787
Epoch #3000 Loss: 0.5647 accuracy: 0.7234 Test loss: 0.6336 Test accuracy: 0.6691 Learning rate: 0.08573749661445618
Epoch #3500 Loss: 0.5703 accuracy: 0.7229 Test loss: 0.6671 Test accuracy: 0.6650 Learning rate: 0.08573749661445618
Epoch #4000 Loss: 0.5297 accuracy: 0.7408 Test loss: 0.5788 Test accuracy: 0.6988 Learning rate: 0.08145061880350113
Epoch #4500 Loss: 0.5649 accuracy: 0.7225 Test loss: 0.5518 Test ac

Epoch #35500 Loss: 0.4142 accuracy: 0.8031 Test loss: 0.5043 Test accuracy: 0.7715 Learning rate: 0.016608331352472305
Epoch #36000 Loss: 0.4180 accuracy: 0.8001 Test loss: 0.5288 Test accuracy: 0.7690 Learning rate: 0.015777913853526115
Epoch #36500 Loss: 0.4130 accuracy: 0.8078 Test loss: 0.5271 Test accuracy: 0.7665 Learning rate: 0.015777913853526115
Epoch #37000 Loss: 0.4151 accuracy: 0.8026 Test loss: 0.5057 Test accuracy: 0.7640 Learning rate: 0.014989017508924007
Epoch #37500 Loss: 0.4104 accuracy: 0.8056 Test loss: 0.5054 Test accuracy: 0.7673 Learning rate: 0.014989017508924007
Epoch #38000 Loss: 0.4126 accuracy: 0.8031 Test loss: 0.5040 Test accuracy: 0.7690 Learning rate: 0.014239566400647163
Epoch #38500 Loss: 0.4153 accuracy: 0.8069 Test loss: 0.5235 Test accuracy: 0.7632 Learning rate: 0.014239566400647163
Epoch #39000 Loss: 0.4108 accuracy: 0.8090 Test loss: 0.5307 Test accuracy: 0.7723 Learning rate: 0.013527587056159973
Epoch #39500 Loss: 0.4151 accuracy: 0.8014 Test 

## Generate missing labels

In [204]:
ones_indices = np.nonzero(labels)
ratio_of_hidden_samples = 0.1
number_of_hidden_samples = int(len(ones_indices[0]) * ratio_of_hidden_samples)
random_indices = random.sample(list(np.arange(len(ones_indices[0]))),number_of_hidden_samples)
indices_to_hide = (ones_indices[0][random_indices] , ones_indices[1][random_indices])
labels_with_missing_positives = np.copy(labels)
for counter in range (number_of_hidden_samples):
    labels_with_missing_positives[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0

In [205]:
# Training with missing labels with 10%
NUM_EPOCHS = 50000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels_with_missing_positives})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), 
                  "Test accuracy: {:.4f}".format(val_accuracies), "Learning rate: {}".format(current_learning_rate))

Epoch #500 Loss: 0.5590 accuracy: 0.7302 Test loss: 0.6049 Test accuracy: 0.6807 Learning rate: 0.10000000149011612
Epoch #1000 Loss: 0.5596 accuracy: 0.7319 Test loss: 0.6122 Test accuracy: 0.6782 Learning rate: 0.0949999988079071
Epoch #1500 Loss: 0.5347 accuracy: 0.7400 Test loss: 0.5758 Test accuracy: 0.6939 Learning rate: 0.0949999988079071
Epoch #2000 Loss: 0.6166 accuracy: 0.6918 Test loss: 0.6890 Test accuracy: 0.6708 Learning rate: 0.09025000035762787
Epoch #2500 Loss: 0.5518 accuracy: 0.7400 Test loss: 0.7069 Test accuracy: 0.6708 Learning rate: 0.09025000035762787
Epoch #3000 Loss: 0.5565 accuracy: 0.7302 Test loss: 0.5926 Test accuracy: 0.7013 Learning rate: 0.08573749661445618
Epoch #3500 Loss: 0.5547 accuracy: 0.7370 Test loss: 0.6173 Test accuracy: 0.6716 Learning rate: 0.08573749661445618
Epoch #4000 Loss: 0.5652 accuracy: 0.7289 Test loss: 0.6149 Test accuracy: 0.6741 Learning rate: 0.08145061880350113
Epoch #4500 Loss: 0.5755 accuracy: 0.7298 Test loss: 0.6273 Test ac

Epoch #35500 Loss: 0.4412 accuracy: 0.7835 Test loss: 0.5162 Test accuracy: 0.7450 Learning rate: 0.016608331352472305
Epoch #36000 Loss: 0.4425 accuracy: 0.7792 Test loss: 0.5196 Test accuracy: 0.7450 Learning rate: 0.015777913853526115
Epoch #36500 Loss: 0.4416 accuracy: 0.7813 Test loss: 0.5180 Test accuracy: 0.7459 Learning rate: 0.015777913853526115
Epoch #37000 Loss: 0.4418 accuracy: 0.7792 Test loss: 0.5187 Test accuracy: 0.7459 Learning rate: 0.014989017508924007
Epoch #37500 Loss: 0.4401 accuracy: 0.7813 Test loss: 0.5164 Test accuracy: 0.7483 Learning rate: 0.014989017508924007
Epoch #38000 Loss: 0.4410 accuracy: 0.7792 Test loss: 0.5192 Test accuracy: 0.7450 Learning rate: 0.014239566400647163
Epoch #38500 Loss: 0.4379 accuracy: 0.7852 Test loss: 0.5149 Test accuracy: 0.7483 Learning rate: 0.014239566400647163
Epoch #39000 Loss: 0.4382 accuracy: 0.7835 Test loss: 0.5174 Test accuracy: 0.7459 Learning rate: 0.013527587056159973
Epoch #39500 Loss: 0.4368 accuracy: 0.7843 Test 

# with 40% hidden labels

In [206]:
ones_indices = np.nonzero(labels)
ratio_of_hidden_samples = 0.4
number_of_hidden_samples = int(len(ones_indices[0]) * ratio_of_hidden_samples)
random_indices = random.sample(list(np.arange(len(ones_indices[0]))),number_of_hidden_samples)
indices_to_hide = (ones_indices[0][random_indices] , ones_indices[1][random_indices])
labels_with_missing_positives = np.copy(labels)
for counter in range (number_of_hidden_samples):
    labels_with_missing_positives[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0
    
    
# Training with missing labels with 40%
NUM_EPOCHS = 50000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels_with_missing_positives})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), 
                  "Test accuracy: {:.4f}".format(val_accuracies), "Learning rate: {}".format(current_learning_rate))

Epoch #500 Loss: 0.4654 accuracy: 0.8188 Test loss: 0.7191 Test accuracy: 0.6708 Learning rate: 0.10000000149011612
Epoch #1000 Loss: 0.4577 accuracy: 0.8184 Test loss: 0.6441 Test accuracy: 0.6716 Learning rate: 0.0949999988079071
Epoch #1500 Loss: 0.4597 accuracy: 0.8171 Test loss: 0.6477 Test accuracy: 0.6708 Learning rate: 0.0949999988079071
Epoch #2000 Loss: 0.4793 accuracy: 0.8188 Test loss: 0.7385 Test accuracy: 0.6708 Learning rate: 0.09025000035762787
Epoch #2500 Loss: 0.4786 accuracy: 0.8193 Test loss: 0.6396 Test accuracy: 0.6724 Learning rate: 0.09025000035762787
Epoch #3000 Loss: 0.4565 accuracy: 0.8193 Test loss: 0.6697 Test accuracy: 0.6708 Learning rate: 0.08573749661445618
Epoch #3500 Loss: 0.4670 accuracy: 0.8201 Test loss: 0.6294 Test accuracy: 0.6708 Learning rate: 0.08573749661445618
Epoch #4000 Loss: 0.4241 accuracy: 0.8193 Test loss: 0.6530 Test accuracy: 0.6724 Learning rate: 0.08145061880350113
Epoch #4500 Loss: 0.4779 accuracy: 0.8193 Test loss: 0.6281 Test ac

Epoch #35500 Loss: 0.3546 accuracy: 0.8372 Test loss: 0.7191 Test accuracy: 0.6997 Learning rate: 0.016608331352472305
Epoch #36000 Loss: 0.3577 accuracy: 0.8393 Test loss: 0.7175 Test accuracy: 0.7120 Learning rate: 0.015777913853526115
Epoch #36500 Loss: 0.3529 accuracy: 0.8376 Test loss: 0.7240 Test accuracy: 0.6988 Learning rate: 0.015777913853526115
Epoch #37000 Loss: 0.3570 accuracy: 0.8363 Test loss: 0.7295 Test accuracy: 0.7038 Learning rate: 0.014989017508924007
Epoch #37500 Loss: 0.3550 accuracy: 0.8359 Test loss: 0.7259 Test accuracy: 0.7087 Learning rate: 0.014989017508924007
Epoch #38000 Loss: 0.3513 accuracy: 0.8384 Test loss: 0.7230 Test accuracy: 0.7021 Learning rate: 0.014239566400647163
Epoch #38500 Loss: 0.3519 accuracy: 0.8389 Test loss: 0.7266 Test accuracy: 0.7038 Learning rate: 0.014239566400647163
Epoch #39000 Loss: 0.3523 accuracy: 0.8376 Test loss: 0.7296 Test accuracy: 0.7096 Learning rate: 0.013527587056159973
Epoch #39500 Loss: 0.3501 accuracy: 0.8410 Test 

## With negative weights (fixed weight) 

In [219]:
train_negative_weights = np.zeros_like(labels) + 1 
train_positive_weights = np.zeros_like(labels) + 1
for counter in range (number_of_hidden_samples):
    train_negative_weights[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0

In [370]:
positive_weights = tf.placeholder(tf.float32, [None,output_shape], name = "Positive_weights")
negative_weights = tf.placeholder(tf.float32, [None, output_shape], name="negative_weights")
my_weights_loss = weighted_loss(y_true= y, y_pred= output,
                              positive_weights= positive_weights, negative_weights= negative_weights)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(my_weights_loss,global_step=global_step)

In [221]:
# Training with negative weights!
NUM_EPOCHS = 50000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_my_weights_loss, epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([my_weights_loss, loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         features,y: labels_with_missing_positives,positive_weights: train_positive_weights,
                                                                                                  negative_weights: train_negative_weights})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "Weighted Loss: {:.4f}".format(epoch_my_weights_loss),"accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), "Test accuracy: {:.4f}".format(val_accuracies))

Epoch #500 Loss: 0.4728 Weighted Loss: 0.4352 accuracy: 0.8184 Test loss: 0.7250 Test accuracy: 0.6716
Epoch #1000 Loss: 0.4488 Weighted Loss: 0.4135 accuracy: 0.8193 Test loss: 0.6207 Test accuracy: 0.6741
Epoch #1500 Loss: 0.5234 Weighted Loss: 0.4958 accuracy: 0.8167 Test loss: 0.6832 Test accuracy: 0.6708
Epoch #2000 Loss: 0.4592 Weighted Loss: 0.4290 accuracy: 0.8176 Test loss: 0.6610 Test accuracy: 0.6700
Epoch #2500 Loss: 0.4515 Weighted Loss: 0.4215 accuracy: 0.8176 Test loss: 0.6453 Test accuracy: 0.6741
Epoch #3000 Loss: 0.4981 Weighted Loss: 0.4696 accuracy: 0.8171 Test loss: 0.7003 Test accuracy: 0.6617
Epoch #3500 Loss: 0.4537 Weighted Loss: 0.4155 accuracy: 0.8184 Test loss: 0.6471 Test accuracy: 0.6889
Epoch #4000 Loss: 0.4405 Weighted Loss: 0.4013 accuracy: 0.8163 Test loss: 0.5821 Test accuracy: 0.6939
Epoch #4500 Loss: 0.4452 Weighted Loss: 0.4089 accuracy: 0.8176 Test loss: 0.5732 Test accuracy: 0.7071
Epoch #5000 Loss: 0.4316 Weighted Loss: 0.3732 accuracy: 0.8014 T

Epoch #40000 Loss: 0.3679 Weighted Loss: 0.2600 accuracy: 0.8380 Test loss: 0.6288 Test accuracy: 0.7327
Epoch #40500 Loss: 0.3557 Weighted Loss: 0.2525 accuracy: 0.8470 Test loss: 0.6319 Test accuracy: 0.7393
Epoch #41000 Loss: 0.3603 Weighted Loss: 0.2531 accuracy: 0.8423 Test loss: 0.6385 Test accuracy: 0.7500
Epoch #41500 Loss: 0.3575 Weighted Loss: 0.2516 accuracy: 0.8402 Test loss: 0.6580 Test accuracy: 0.7401
Epoch #42000 Loss: 0.3588 Weighted Loss: 0.2542 accuracy: 0.8453 Test loss: 0.6394 Test accuracy: 0.7368
Epoch #42500 Loss: 0.3542 Weighted Loss: 0.2487 accuracy: 0.8474 Test loss: 0.6380 Test accuracy: 0.7393
Epoch #43000 Loss: 0.3632 Weighted Loss: 0.2540 accuracy: 0.8406 Test loss: 0.6487 Test accuracy: 0.7327
Epoch #43500 Loss: 0.3573 Weighted Loss: 0.2487 accuracy: 0.8440 Test loss: 0.6436 Test accuracy: 0.7343
Epoch #44000 Loss: 0.3584 Weighted Loss: 0.2485 accuracy: 0.8419 Test loss: 0.6480 Test accuracy: 0.7327
Epoch #44500 Loss: 0.3560 Weighted Loss: 0.2475 accurac

## Computing correlations for negative weights

In [277]:
def negative_labeles_probabilities(hot_encoded,LABELS_LIST):
    # count the number of times a combination has appeared with the negative label as 0 / the total number of
    # occurances of that combination without the negative label
    negative_weights = np.zeros([len(hot_encoded), len(LABELS_LIST)])
    for sample_idx in range(len(hot_encoded)):
        for label_idx in range(len(LABELS_LIST)):
            if hot_encoded.iloc[sample_idx, label_idx] == 1:
                negative_weights[sample_idx, label_idx] = 0
            else:
                temp_combination = hot_encoded.iloc[sample_idx,:].copy()
                temp_combination[label_idx] = 1
                positive_samples = len(hot_encoded[(hot_encoded.iloc[:, :].values == temp_combination.values).all(axis = 1)])
                negative_samples = len(hot_encoded[(hot_encoded.iloc[:, :].values == hot_encoded.iloc[sample_idx, :].values).all(axis=1)])
                negative_weights[sample_idx, label_idx] = negative_samples / (positive_samples + negative_samples)
    negative_weights_df = pd.DataFrame(negative_weights, columns=LABELS_LIST)
    #negative_weights_df["song_id"] = hot_encoded.song_id
    #negative_weights_df = negative_weights_df[["song_id"] + LABELS_LIST]
    #negative_weights_df.to_csv("/home/karim/Documents/BalancedDatasetDeezer/GroundTruth/negative_weights.csv",index=False)
    return negative_weights_df

In [279]:
negative_weights_df = negative_labeles_probabilities(pd.DataFrame(labels_with_missing_positives), LABELS_LIST = LABEL_LIST)
hidden_weights = []
for x in range(len(indices_to_hide[0])):
        hidden_weights.append(negative_weights_df.values[indices_to_hide[0][x]][indices_to_hide[1][x]])
        
correlations_based_weights = negative_weights_df.values

In [332]:
def negative_labels_probabilities_pairwise(hot_encoded,LABELS_LIST):
    # count the number of times a combination has appeared with the negative label as 1 / the total number of
    # occurances of that combination without the negative label
    negative_weights = np.ones([len(hot_encoded), len(LABELS_LIST)])
    for sample_idx in range(len(hot_encoded)):
        for label_idx in range(len(LABELS_LIST)):
            if hot_encoded.iloc[sample_idx, label_idx+1] == 1:
                negative_weights[sample_idx, label_idx] = 0
            else:
                for other_labels_index in range(len(LABELS_LIST)):
                    # Iterate through all other labels for each label_index
                    # check patterns if the other label == 1 (weight is #times occured as 0 / number of times occured as 0 or 1 for the target label)
                    if (other_labels_index != label_idx) and (hot_encoded.iloc[sample_idx, other_labels_index] == 1):
                        positive_occurances = len(hot_encoded[(hot_encoded.iloc[:, [label_idx,other_labels_index]].values == [1,1]).all(axis = 1)])
                        negative_occurances = len(hot_encoded[(hot_encoded.iloc[:, [label_idx,other_labels_index]].values == [0,1]).all(axis = 1)])
                        weight = negative_occurances / (negative_occurances + positive_occurances)
                    # Check the patterns if the other labels == 0
                    if (other_labels_index != label_idx) and (hot_encoded.iloc[sample_idx, other_labels_index] == 0):
                        positive_occurances = len(hot_encoded[(hot_encoded.iloc[:, [label_idx,other_labels_index]].values == [1,0]).all(axis = 1)])
                        negative_occurances = len(hot_encoded[(hot_encoded.iloc[:, [label_idx,other_labels_index]].values == [0,0]).all(axis = 1)])
                        weight = negative_occurances / (negative_occurances + positive_occurances)
                    negative_weights[sample_idx, label_idx] += weight
                negative_weights[sample_idx, label_idx] /= (len(LABELS_LIST))
    negative_weights_df = pd.DataFrame(negative_weights, columns=LABELS_LIST)
    negative_weights_df["song_id"] = hot_encoded.song_id
    negative_weights_df = negative_weights_df[["song_id"] + LABELS_LIST]
    #negative_weights_df.to_csv("/home/karim/Documents/BalancedDatasetDeezer/GroundTruth/negative_weights_paris.csv",index=False)
    return negative_weights_df

In [371]:
# Training with negative weights!
NUM_EPOCHS = 50000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_my_weights_loss, epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([my_weights_loss, loss, accuracy,output, train_step],feed_dict={x_input: features,
                                                                                                                                                     y: labels_with_missing_positives,positive_weights: train_positive_weights,
                                                                                                                                                     negative_weights: correlations_based_weights})
        if (epoch+1)% 500 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels})
            print("Epoch #{}".format(epoch+1), "Loss: {:.4f}".format(epoch_loss), 
                  "Weighted Loss: {:.4f}".format(epoch_my_weights_loss),"accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), "Test accuracy: {:.4f}".format(val_accuracies))

Epoch #500 Loss: 0.4679 Weighted Loss: 0.4345 accuracy: 0.8188 Test loss: 0.6502 Test accuracy: 0.6708
Epoch #1000 Loss: 0.4559 Weighted Loss: 0.4248 accuracy: 0.8180 Test loss: 0.6336 Test accuracy: 0.6716
Epoch #1500 Loss: 0.4920 Weighted Loss: 0.4622 accuracy: 0.8163 Test loss: 0.6275 Test accuracy: 0.6757
Epoch #2000 Loss: 0.4430 Weighted Loss: 0.4061 accuracy: 0.8214 Test loss: 0.6879 Test accuracy: 0.6733
Epoch #2500 Loss: 0.4445 Weighted Loss: 0.4092 accuracy: 0.8099 Test loss: 0.6258 Test accuracy: 0.6955
Epoch #3000 Loss: 0.4175 Weighted Loss: 0.3821 accuracy: 0.8210 Test loss: 0.6289 Test accuracy: 0.6823
Epoch #3500 Loss: 0.4351 Weighted Loss: 0.3979 accuracy: 0.8159 Test loss: 0.6851 Test accuracy: 0.6741
Epoch #4000 Loss: 0.4168 Weighted Loss: 0.3860 accuracy: 0.8257 Test loss: 0.5865 Test accuracy: 0.6931
Epoch #4500 Loss: 0.4123 Weighted Loss: 0.3815 accuracy: 0.8248 Test loss: 0.5864 Test accuracy: 0.6898
Epoch #5000 Loss: 0.4067 Weighted Loss: 0.3720 accuracy: 0.8227 T

Epoch #40000 Loss: 0.2875 Weighted Loss: 0.2535 accuracy: 0.8679 Test loss: 0.7986 Test accuracy: 0.7186
Epoch #40500 Loss: 0.2952 Weighted Loss: 0.2605 accuracy: 0.8589 Test loss: 0.8164 Test accuracy: 0.7178
Epoch #41000 Loss: 0.2872 Weighted Loss: 0.2519 accuracy: 0.8717 Test loss: 0.7530 Test accuracy: 0.7294
Epoch #41500 Loss: 0.2850 Weighted Loss: 0.2518 accuracy: 0.8674 Test loss: 0.7288 Test accuracy: 0.7285
Epoch #42000 Loss: 0.2843 Weighted Loss: 0.2510 accuracy: 0.8725 Test loss: 0.7972 Test accuracy: 0.7219
Epoch #42500 Loss: 0.2823 Weighted Loss: 0.2487 accuracy: 0.8632 Test loss: 0.7743 Test accuracy: 0.7236
Epoch #43000 Loss: 0.2950 Weighted Loss: 0.2599 accuracy: 0.8581 Test loss: 0.8437 Test accuracy: 0.7170
Epoch #43500 Loss: 0.2828 Weighted Loss: 0.2483 accuracy: 0.8725 Test loss: 0.7963 Test accuracy: 0.7203
Epoch #44000 Loss: 0.2806 Weighted Loss: 0.2470 accuracy: 0.8717 Test loss: 0.7428 Test accuracy: 0.7244
Epoch #44500 Loss: 0.2794 Weighted Loss: 0.2452 accurac

## Evaluation

In [389]:
# On test
print("AUC is:{:.3f}".format(roc_auc_score(test_labels, val_output)))
print("F1 is:{:.3f}".format(f1_score(test_labels, np.round(val_output),average="macro")))
print("Recall is:{:.3f}".format(recall_score(test_labels, np.round(val_output),average="macro")))
print("Precision is:{:.3f}".format(precision_score(test_labels, np.round(val_output),average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(test_labels, np.round(val_output))))

AUC is:0.737
F1 is:0.450
Recall is:0.350
Precision is:0.653
Hamming loss is:0.276


In [391]:
# On training 
print("AUC is:{:.3f}".format(roc_auc_score(labels, epoch_output)))
print("F1 is:{:.3f}".format(f1_score(labels, np.round(epoch_output),average="macro")))
print("Recall is:{:.3f}".format(recall_score(labels, np.round(epoch_output),average="macro")))
print("Precision is:{:.3f}".format(precision_score(labels, np.round(epoch_output),average="macro")))
print("Hamming loss is:{:.3f}".format(hamming_loss(labels, np.round(epoch_output))))

AUC is:0.868
F1 is:0.572
Recall is:0.450
Precision is:0.872
Hamming loss is:0.185
